# $Z_{20}$ vs ssh

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import copy
import time

# Import custom modules
import src.utils

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## funcs

## Load data

In [ ]:
## load spatial data
forced, anom = src.utils.load_consolidated()

## subset
anom = anom[["ssh", "ssh_comp", "z20", "z20_comp", "nhf", "nhf_comp"]]

Data seems to not be aligned...check this:

## Compute indices

In [ ]:
## times not aligned??
# anom["z20"].isel(time=0).values

In [ ]:
idxs = src.utils.reconstruct_wrapper(
    anom,
    fn=src.utils.get_RO_hw,
)

idxs = src.utils.get_windowed(idxs)
idxs0 = idxs.isel(year=0).transpose("time", "member")

Look at correlation with NHF

In [ ]:
import scipy.stats

r = lambda x, y: scipy.stats.pearsonr(x, y)[0].item()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(
    idxs0["ssh"].values,
    idxs0["z20"].values,
    s=1,
)